# TIR 入门

参考：[TensorIR 的突击课程](https://daobook.github.io/tvm/docs/tutorial/tensor_ir_blitz_course.html)

In [1]:
import tvm
from tvm.ir.module import IRModule
from tvm.script import tir as T
import numpy as np


@tvm.script.ir_module
class MyModule:
    @T.prim_func
    def main(a: T.handle, b: T.handle):
        # 通过句柄在函数之间交换数据，这类似于指针。
        T.func_attr({"global_symbol": "main", "tir.noalias": True})
        # 从句柄创建缓冲区。
        A = T.match_buffer(a, (8,), dtype="float32")
        B = T.match_buffer(b, (8,), dtype="float32")
        for i in range(8):
            # 块是计算的抽象。
            with T.block("B"):
                # 定义空间块迭代器，并将其绑定到值 i。
                vi = T.axis.spatial(8, i)
                B[vi] = A[vi] + 1.0


ir_module = MyModule
ir_module.show()

@tvm.script.ir_module
class Module:
    @T.prim_func
    def main(A: T.Buffer[8, "float32"], B: T.Buffer[8, "float32"]) -> None:
        # function attr dict
        T.func_attr({"global_symbol": "main", "tir.noalias": True})
        # body
        # with T.block("root")
        for i in T.serial(8):
            with T.block("B"):
                vi = T.axis.spatial(8, i)
                T.reads(A[vi])
                T.writes(B[vi])
                B[vi] = A[vi] + T.float32(1)
    



In [4]:
tvm.relax

<module 'tvm.relax' from '/media/pc/data/4tb/lxw/libs/anaconda3/envs/py38/lib/python3.8/site-packages/tvm/relax/__init__.py'>

- `@tvm.script.ir_module`：表示被其 annotate 的类别 MyModule，就是一个待编译的 IRModule
- `@T.prim_func`：表示被其 annotate 的成员函数 main，就是 IRModule 的一个 PrimFunc

`script` 中 `T.*` 的部分，就对应着 AST 中的树节点。

编译：

In [ ]:
import numpy as np

# mod = tvm.build(ir_module, target="c")
mod = tvm.build(ir_module, target="llvm")
# mod = tvm.build(ir_module, target="cuda")

a = tvm.nd.array(np.arange(8).astype("float32"))
print(a)
# [0. 1. 2. 3. 4. 5. 6. 7.]

b = tvm.nd.array(np.zeros((8,)).astype("float32"))
mod(a, b)
print(b)

`tvm.build` 的最后一个参数 `target`，就是用来选择用哪一个 CodeGen 来编译 TIR AST。

例如，如果要编译为 CPU 运行的代码，那么参数可以是 `target="c"`，也可以是 `target="llvm"`；如果要编译为 GPU 运行的代码，那么参数是 `target="cuda"`。

`tvm.build` 会根据 `target` 参数，寻找已经注册的编译函数。在 TVM 中，用宏定义 `TVM_REGISTER_GLOBAL` 注册编译函数：

```c++
// src/target/source/codegen_c_host.cc
TVM_REGISTER_GLOBAL("target.build.c").set_body_typed(BuildCHost);

// src/target/opt/build_cuda_on.cc
TVM_REGISTER_GLOBAL("target.build.cuda").set_body_typed(BuildCUDA);

// src/target/llvm/llvm_module.cc
TVM_REGISTER_GLOBAL("target.build.llvm")
    .set_body_typed([](IRModule mod, Target target) -> runtime::Module {
      auto n = make_object<LLVMModuleNode>();
      n->Init(mod, target);
      return runtime::Module(n);
    })
```

对于 C++ 和 CUDA，`tvm.build` 有两个步骤：

```
TIR -> C++/CUDA -> bin
```

先通过相应的 CodeGen，生成源代码；然后调用相应的编译器，生成可执行文件并且打包为 runtime。

如果 `target="llvm"`，由于 LLVM IR 仍然只是一种中间表示，还需要根据 `target` 当中更详细的硬件参数，找到目标编译硬件，然后调用相应的 CodeGen（省略部分辅助代码）：

```c++
void Init(const IRModule& mod, const Target& target) {
  // Step 1: Initialize CodeGen for LLVM with different target
  InitializeLLVM();
  tm_ = GetLLVMTargetMachine(target);
  std::unique_ptr<CodeGenLLVM> cg = CodeGenLLVM::Create(tm_.get());

  // Step 2: Add all tir::PrimFunc in IRModule to compile list
  std::vector<PrimFunc> funcs;
  for (auto kv : mod->functions) {
    if (!kv.second->IsInstance<PrimFuncNode>()) {
      // (@jroesch): we relax constraints here, Relay functions will just be ignored.
      DLOG(INFO) << "Can only lower IR Module with PrimFuncs, but got "
                  << kv.second->GetTypeKey();
      continue;
    }
    auto f = Downcast<PrimFunc>(kv.second);
    funcs.push_back(f);
  }

  // Step 3: Lower IRModule to LLVM IR code
  module_ = cg->Finish();
}
```



此外，还可以使用张量表达式 DSL（domain-specific language）来编写简单的算子，并将其转换为 IRModule。

In [ ]:
from tvm import te

A = te.placeholder((8,), dtype="float32", name="A")
B = te.compute((8,), lambda *i: A(*i) + 1.0, name="B")
func = te.create_prim_func([A, B])
ir_module_from_te = IRModule({"main": func})
ir_module_from_te.show()

TIR 能 lower 成目标源代码，关键是 CodeGen。上面提到的 CodeGenCHost，以及 CodeGenCUDA，都是继承自 CodeGenC，即将 TIR lower 为 C++ 代码。

因为 TIR AST 是 Graph 结构（Tree 也是一种特殊的树），因此 CodeGenC 根本上是一个 Graph 遍历器。当 CodeGenC 遍历到某个 TIR Node 的时候，根据 TIR Node 的类型和属性，翻译为相应的 C++ 代码。下面是 CodeGenC 的部分定义，位于 `tvm/src/target/source/codegen_c.[h, cc]` 中：

```c++
class CodeGenC : public ExprFunctor<void(const PrimExpr&, std::ostream&)>,
                 public StmtFunctor<void(const Stmt&)>,
                 public CodeGenSourceBase {
 public:
  // expression
  void VisitExpr_(const VarNode* op, std::ostream& os) override;         // NOLINT(*)
  void VisitExpr_(const LoadNode* op, std::ostream& os) override;        // NOLINT(*)
  void VisitExpr_(const BufferLoadNode* op, std::ostream& os) override;  // NOLINT(*)
  void VisitExpr_(const LetNode* op, std::ostream& os) override;         // NOLINT(*)
  void VisitExpr_(const CallNode* op, std::ostream& os) override;        // NOLINT(*)
  void VisitExpr_(const AddNode* op, std::ostream& os) override;         // NOLINT(*)
  void VisitExpr_(const SubNode* op, std::ostream& os) override;         // NOLINT(*)
  void VisitExpr_(const MulNode* op, std::ostream& os) override;         // NOLINT(*)
  void VisitExpr_(const DivNode* op, std::ostream& os) override;         // NOLINT(*)
  void VisitExpr_(const ModNode* op, std::ostream& os) override;         // NOLINT(*)
  void VisitExpr_(const MinNode* op, std::ostream& os) override;         // NOLINT(*)
  void VisitExpr_(const MaxNode* op, std::ostream& os) override;         // NOLINT(*)
  void VisitExpr_(const EQNode* op, std::ostream& os) override;          // NOLINT(*)
  void VisitExpr_(const NENode* op, std::ostream& os) override;          // NOLINT(*)
  void VisitExpr_(const LTNode* op, std::ostream& os) override;          // NOLINT(*)
  void VisitExpr_(const LENode* op, std::ostream& os) override;          // NOLINT(*)
  void VisitExpr_(const GTNode* op, std::ostream& os) override;          // NOLINT(*)
  void VisitExpr_(const GENode* op, std::ostream& os) override;          // NOLINT(*)
  void VisitExpr_(const AndNode* op, std::ostream& os) override;         // NOLINT(*)
  void VisitExpr_(const OrNode* op, std::ostream& os) override;          // NOLINT(*)
  void VisitExpr_(const CastNode* op, std::ostream& os) override;        // NOLINT(*)
  void VisitExpr_(const NotNode* op, std::ostream& os) override;         // NOLINT(*)
  void VisitExpr_(const SelectNode* op, std::ostream& os) override;      // NOLINT(*)
  void VisitExpr_(const RampNode* op, std::ostream& os) override;        // NOLINT(*)
  void VisitExpr_(const ShuffleNode* op, std::ostream& os) override;     // NOLINT(*)
  void VisitExpr_(const BroadcastNode* op, std::ostream& os) override;   // NOLINT(*)
  void VisitExpr_(const IntImmNode* op, std::ostream& os) override;      // NOLINT(*)
  void VisitExpr_(const FloatImmNode* op, std::ostream& os) override;    // NOLINT(*)
  void VisitExpr_(const StringImmNode* op, std::ostream& os) override;   // NOLINT(*)
  // statment
  void VisitStmt_(const LetStmtNode* op) override;
  void VisitStmt_(const StoreNode* op) override;
  void VisitStmt_(const BufferStoreNode* op) override;
  void VisitStmt_(const ForNode* op) override;
  void VisitStmt_(const WhileNode* op) override;
  void VisitStmt_(const IfThenElseNode* op) override;
  void VisitStmt_(const AllocateNode* op) override;
  void VisitStmt_(const AttrStmtNode* op) override;
  void VisitStmt_(const AssertStmtNode* op) override;
  void VisitStmt_(const EvaluateNode* op) override;
  void VisitStmt_(const SeqStmtNode* op) override;
  void VisitStmt_(const AllocateConstNode* op) override;
}
```

可以看到，CodeGenC 会遍历到两种 TIR Node：Expression（表达式） 和 Statement（语句）。Expression（表达式）中包含了常见的变量声明、运算、判断、函数调用，而 Statement（语句）中包含了控制流（if-else，Loop 等）、内存管理、赋值等操作。

例如，遇到四则运算的 Expression，CodeGenC 直接翻译为 " a OP b "的代码：

```c++
template <typename T>
inline void PrintBinaryExpr(const T* op, const char* opstr,
                            std::ostream& os, CodeGenC* p) {
  // If both a and b are scalars
  if (op->dtype.lanes() == 1) {
    // If OP is an alphabet string, then lower it as "OP(a, b)"
    if (isalpha(opstr[0])) {
      os << opstr << '(';
      p->PrintExpr(op->a, os);
      os << ", ";
      p->PrintExpr(op->b, os);
      os << ')';
    }
    // If OP is a symbol, like + - * / %, then lower it as "a OP b"
    else {
      os << '(';
      p->PrintExpr(op->a, os);
      os << ' ' << opstr << ' ';
      p->PrintExpr(op->b, os);
      os << ')';
    }
  }
  // If both a and b are vectors
  else {
    p->PrintVecBinaryOp(opstr, op->dtype, op->a, op->b, os);
  }
}

void CodeGenC::VisitExpr_(const AddNode* op, std::ostream& os) {  // NOLINT(*)
  PrintBinaryExpr(op, "+", os, this);
}
void CodeGenC::VisitExpr_(const SubNode* op, std::ostream& os) {  // NOLINT(*)
  PrintBinaryExpr(op, "-", os, this);
}
void CodeGenC::VisitExpr_(const MulNode* op, std::ostream& os) {  // NOLINT(*)
  PrintBinaryExpr(op, "*", os, this);
}
void CodeGenC::VisitExpr_(const DivNode* op, std::ostream& os) {  // NOLINT(*)
  PrintBinaryExpr(op, "/", os, this);
}
```

如果遇到选择 SelectNode，CodeGenC 则翻译为 "(c ? a : b)" 的代码：

```c++
void CodeGenC::VisitExpr_(const SelectNode* op, std::ostream& os) {
  os << "(";
  PrintExpr(op->condition, os);
  os << " ? ";
  PrintExpr(op->true_value, os);
  os << " : ";
  PrintExpr(op->false_value, os);
  os << ")";
}
```

如果遇到 ForNode，CodeGenC 则翻译为

```
for (DTYPE VID = 0; VID < EXTEND; ++VID) {
BODY
}\n
```

的代码：

```c++
void CodeGenC::VisitStmt_(const ForNode* op) {
  std::string extent = PrintExpr(op->extent);
  PrintIndent();
  std::string vid = AllocVarID(op->loop_var.get());
  ICHECK(is_zero(op->min));
  stream << "for (";
  PrintType(op->loop_var.dtype(), stream);
  stream << ' ' << vid << " = 0; " << vid << " < " << extent << "; ++" << vid << ") {\n";
  int for_scope = BeginScope();
  PrintStmt(op->body);
  this->EndScope(for_scope);
  PrintIndent();
  stream << "}\n";
}
```

